In [ ]:
# initiates a statevector in state |00..0>
def initiate(n):
    'n : number of qubits/fermions (int)'
    sv = np.zeros(2**n)
    sv[0]=1
    return sv.T

'''Random Quantum Circuits'''
# one block of random unitary matrix, outputs the unitary matrix
def random_block(alpha,params,N):
    params = np.pi*(abs(params)/(np.max(abs(params))+np.min(abs(params))))
    iden = 'I'
    for i in range(N-3):
        iden+='I'
    
    terms = []
    for i in range(N-1):
        terms.append(scipy.linalg.expm(-1j*(np.pi/4)*operator(iden[:i]+alpha+alpha+iden[i:])))
    terms.append(scipy.linalg.expm(-1j*(np.pi/4)*operator(alpha+iden+alpha)))
    
    t = []
    iden+='I'
    for i in range(N):
        terms.append(scipy.linalg.expm(-1j*(np.pi/4)*operator(iden[:i]+'Z'+iden[i:])))
    for i in range(N):
        terms.append(scipy.linalg.expm(-1j*(params[N-(i+1)])*operator(iden[:i]+'X'+iden[i:])))
    
    mat = terms[0]
    for i in range(1,len(terms)):
        mat = np.matmul(terms[i],mat)
    return mat

def random_trajectory(c,var,n):
    mat = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            mat[i,j] = c*np.exp(-1*((i-j)**2)/2*(var**2))
    Q, D = sympy.Matrix(mat).diagonalize()
    M = np.matmul(np.array(Q).astype('float64'),scipy.linalg.sqrtm(np.array(D).astype('float64')))
    return M

'''Observables'''
def magnetization(n):
    z = 'Z'
    for i in range(n-1):
        z+='Z'
    return (1/n)*operator(z)

def expectation_list(observable,states):
    values = []
    for i in range(len(states[0])):
        value = []
        for j in range(len(states)):
            value.append(expectation(observable,states[j][i]))
        values.append((1/len(states))*sum(np.array(value)))
    return values

def E_entropy(states,n):
    values = []
    n = int(n/2)
    for i in range(len(states[0])):
        value = []
        for j in range(len(states)):
            value.append(entanglement_entropy(states[j][i],[k for k in range(n)]))
        values.append((1/len(states))*sum(np.array(value)))
    return values

def first_order(alpha,n):
    mat = np.zeros((2**n,2**n))
    term = 'I'
    for i in range(n-2):
        term+='I'
    for i in range(n):
        mat+=operator(term[:i]+alpha+term[i:])
    return (1/n)*mat

In [ ]:
num_qubits = 8
P = 40
num_realizations = 250
steps = [i for i in range(40)]
statevectors = []
parameters = []

In [ ]:
start = time.time()
for gate in ['X','Z']:
    sublist = []
    parameter = []
    
    # number of realizations
    for j in range(num_realizations):
        ini = initiate(num_qubits)
        coeffs = random_trajectory(np.random.randint(1,6),np.random.rand(1)[0],num_qubits)
        steps = []
        values = []
        pars = []
        
        # number of random unitary blocks
        for i in range(P):
            params = np.matmul(coeffs,np.random.normal(0,1,num_qubits))
            unitary = random_block(gate,params,num_qubits)
            ini = np.matmul(unitary,ini)
            
            pars.append(params)
            steps.append(i)
            values.append(ini)
        
        sublist.append(values)
        parameter.append(pars)
        
        print("Realization :",j+1)
        
    statevectors.append(sublist)
    parameters.append(parameter)

end = time.time()
end-start

In [ ]:
x_states = np.array(load('1000-X8'))
z_states = np.array(load('1000-Z8'))

x_params = np.array(load('Params-X8'))
z_params = np.array(load('Params-Z8'))

In [ ]:
# generating target data
magnetization_X = []
magnetization_Z = []

spin_operator_X = []
spin_operator_Z = []

shape = x_states.shape

spin_operator = first_order('Z',8)
magnet = magnetization(8)

for i in range(shape[0]):
    sublist = [[],[],[],[]]
    for j in range(shape[1]):
        sublist[0].append(expectation(magnet,x_states[i][j]))
        sublist[1].append(expectation(magnet,z_states[i][j]))
        sublist[2].append(expectation(spin_operator,x_states[i][j]))
        sublist[3].append(expectation(spin_operator,z_states[i][j]))
    
    magnetization_X.append(sublist[0])
    magnetization_Z.append(sublist[1])
    spin_operator_X.append(sublist[2])
    spin_operator_Z.append(sublist[3])

In [ ]:
'''The 1D Ising Spin Chain in Transverse Field'''
# gives the matrix representation of the Hamiltonian of Ising Model of provided parameters
def Ising_Model(J,h,N):
    terms = []
    for i in range(N-1):
        terms.append((i,i+1))
    terms.append((0,N-1))
    
    iden = 'I'
    for i in range(N-2):
        iden += 'I'
        
    Hj = np.zeros((2**N,2**N), dtype=complex)
    for i,j in terms:
        Hj = Hj + (-1)*J*operator(iden[:i]+'Z'+iden[i+1:j]+'Z'+iden[j:])
    
    iden+='I'
    Hh = np.zeros((2**N,2**N),dtype=complex)
    for i in range(N):
        Hh = Hh + (-1)*h*operator(iden[:i]+'X'+iden[i+1:])
    return np.real((Hj+Hh))

In [ ]:
def GaussianKernel(x,xd):
    def Gaussian(x,xd):
        N = len(x)
        loss = 0
        for i in range(N):
            for j in range(N):
                loss += (x[i]-xd[j])**2
            
        gamma = (N**2)/(loss)
        return np.exp((-1)*(gamma)*np.sum((x-xd)**2))
    return Gaussian(x,xd)/((Gaussian(x,x)*Gaussian(xd,xd))**0.5)

def matrix_K(params,lamda):
    N = len(params)
    kernel = []
    for i in range(N):
        terms = []
        for j in range(N):
            terms.append(GaussianKernel(params[i],params[j]))
        kernel.append(terms)
    kernel = np.array(kernel) + lamda*np.eye(N)
    return np.linalg.inv(kernel)

def kappa(x,l,params):
    N = len(params)
    K = matrix_K(params,0)
    terms = 0
    for ld in range(N):
        terms += GaussianKernel(x,params[ld])*K[ld,l]
    return terms

def predicted(x,params,states):
    N = len(params)
    pred = 0
    for l in range(N):
        pred += kappa(x,l,params)*states[l]
    return pred

In [ ]:
def NTKernel(network, x1, x2):
    
    x1 = torch.tensor(np.array(x1)).float()
    x2 = torch.tensor(np.array(x2)).float()
    func, params = make_functional(network)
    
    def function(params, x):
        return func(params, x.unsqueeze(0)).squeeze(0)
    
    jac1 = vmap(jacrev(function), (None, 0))(params, x1)
    jac1 = [j.flatten(2) for j in jac1]
    
    jac2 = vmap(jacrev(function), (None, 0))(params, x2)
    jac2 = [j.flatten(2) for j in jac2]
    
    result = torch.stack([torch.einsum('Naf,Mbf->NMab', j1, j2) for j1, j2 in zip(jac1, jac2)])
    result = result.sum(0)
    result = result.detach().numpy().reshape(len(x1),len(x2))
    return result

In [ ]:
def kernel_K(network, params):
    kernel = []
    n = len(params[0])
    params = torch.tensor(params).float().reshape(len(params),1,n)
    for l in range(len(params)):
        terms = []
        for ld in range(len(params)):
            terms.append(np.linalg.inv(NTKernel(network, params[l], params[ld])))
        kernel.append(terms)
    return np.array(kernel).reshape(len(params),len(params))

K = kernel_K(network, params)

In [ ]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
num_epochs = 100
for epoch in range(num_epochs):
    outputs = model(x)
    loss = criterion(outputs, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, loss.item()))

In [ ]:
def prediction(network, xinput, params, states, kernel):
    
    def kernel_k(network, xinput, params):
        N = len(params)
        n = len(params[0])
        kernel = []
        x = torch.tensor(np.array(xinput)).float().reshape(1,n)
        for l in range(N):
            param = torch.tensor(params[l]).float().reshape(1,n)
            kernel.append(NTKernel(network,x,param))
        return np.array(kernel).reshape(len(params),1)
    
    N = len(params)
    n = len(params[0])
    k = kernel_k(network, xinput, params)
    pred = 0
    for l in range(N):
        for ld in range(N):
            state = np.array(states[ld])
            term = k[l]*kernel[l,ld]*state
            pred+=term
    return (1/N)*(1/N)*pred